### SocialAL Models
# Calc llh from gain-loss learning model for subset of data
KLS 12.19.22
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [2]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [3]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools
import os
import csv

## Pull in functions

In [4]:
run common_functions.ipynb

In [5]:
run double_alpha_functions.ipynb

### Pull in parameters fit to Trustworthy/Untrustworthy conditions

In [15]:
pt = pd.read_csv('../../output/generalization_criterion/a2_fit2_2cond_part.csv')
pt.head(5)

id  alpha_gain  alpha_loss          beta        llh
0  sub-2013    0.002731    0.002246  2.000000e+01  32.366876
1  sub-2007    0.247394    0.200707  3.039108e-01  35.884138
2  sub-2006    0.000000    0.000000  1.000000e-10  41.588831
3  sub-2012    0.050043    1.000000  3.805525e-01  32.662632
4  sub-2004    1.000000    0.088426  2.279855e+00  12.709590

### Load and clean data

In [8]:
files = os.listdir('../../data/modeling')
files.remove('.DS_Store')
files.remove('missing')
#print(files)

In [9]:
n_part = len(files)
print('number of participants: ', n_part)

number of participants:  63


In [10]:
def load_and_clean(file):
    path = os.path.join('../../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    # limit to neutral trials only
    data = data[data['Stim_Sequence'] == 1]
    data.reset_index(drop=True, inplace=True)
    return(data)

In [11]:
# data = load_and_clean(files[0])
# data.head(20) # should only have 15 trials

In [46]:
# create csv file
file = open('../../output/generalization_criterion/a2_llh4_1cond_part.csv', 'w')

# add header
headerList = ['id','a2_llh']
with open('../../output/generalization_criterion/a2_llh4_1cond_part.csv', 'a') as f_object: 
    dw = csv.DictWriter(f_object, delimiter = ',', fieldnames = headerList)
    dw.writeheader()

#loop thru participants
for x in range(n_part):
    print('loop:', x)
    subject = files[x][:-4]
    print(subject)
    
    # load and clean data
    print('loading data')
    data = load_and_clean(files[x])
    
    # pull parameters
    print('pull parameters')
    params = pt.loc[pt['id'] == subject].values.flatten().tolist()
    #print(params[1:4])
    
    # calculting llh
    print('calculating llh')
    llh = get_likelihood_action(params[1:4], data)
    a2 = [subject, llh]
    print(a2)
    
    # save results
    print('saving results')
    with open('../../output/generalization_criterion/a2_llh4_1cond_part.csv', 'a') as f_object: # create a file object
        writer_object = csv.writer(f_object) # pass the file object to csv writer
        writer_object.writerow(a2) #pass list as argument to writerow
        f_object.close() # close the file object

loop: 0
sub-2013
loading data
pull parameters
calculating llh
['sub-2013', 14.371683837121704]
saving results
loop: 1
sub-2007
loading data
pull parameters
calculating llh
['sub-2007', 15.708927587769018]
saving results
loop: 2
sub-2006
loading data
pull parameters
calculating llh
['sub-2006', 19.408121055678468]
saving results
loop: 3
sub-2012
loading data
pull parameters
calculating llh
['sub-2012', 15.013056383676123]
saving results
loop: 4
sub-2004
loading data
pull parameters
calculating llh
['sub-2004', 73.8832303728022]
saving results
loop: 5
sub-2010
loading data
pull parameters
calculating llh
['sub-2010', 20.783823024228585]
saving results
loop: 6
sub-2038
loading data
pull parameters
calculating llh
['sub-2038', 20.871048322060357]
saving results
loop: 7
sub-2039
loading data
pull parameters
calculating llh
['sub-2039', 1716.25849995194]
saving results
loop: 8
sub-2011
loading data
pull parameters
calculating llh
['sub-2011', 21.19645265691952]
saving results
loop: 9
sub-200